## Loading Packages

In [ ]:
# import warnings
# warnings.filterwarnings("ignore") # suppress warnings

In [ ]:
import os
import sys

## Clip path to all helper functions

In [ ]:
function_path = os.path.expanduser("~/geoscience/albedo_downscaling/functions")
sys.path.append(function_path)
# import all the helper functions.
from modis_bluesky_albedo import *

In [ ]:
black_sky_albedo_arr =  get_albedo_values(modis_albedo_data_dir, True) # Clear sky
white_sky_albedo_arr =  get_albedo_values(modis_albedo_data_dir, False) # Cloudy sky

In [ ]:
for date, albedo_val in white_sky_albedo_arr.items():
    albedo_val.plot()
    plt.show()

In [ ]:
aod_arr = get_aod_values(aod_data_dir)
print("Done!!!")

## Calculate diffuse skylight ratio lookup table

In [ ]:
# Index lookup table by solar zenith angle (row) and aerosol optical depth (column)
lookup_table = pd.read_csv(diffuse_skylight_ratio_lookup)

# Ensure that the types match
print("Type of index values:", type(lookup_table.index[0]))
print("Type of column values:", type(lookup_table.columns[0]))

In [ ]:
# print(converted_aod_value, type(converted_aod_value))
print(lookup_table.columns[1])

## Calculate Blue Sky Albedo

In [ ]:
blue_sky_albedo = calculate_blue_sky_albedo(black_sky_albedo_arr, white_sky_albedo_arr, lookup_table, ex_format_tif)

In [ ]:
print(len(blue_sky_albedo))

## Write Blue Sky Albedo Data to Raster

In [ ]:
# Define naming convention for output files
name_format = "modis_blue_sky_albedo_new"

# Loop through the blue sky albedo dictionary, which is indexed by date
for date, value in blue_sky_albedo.items():
    # Construct output filename using the date and naming convention
    new_file_name = f"{date}_{name_format}.tif"
    
    # Define output directory path for saving the raster files
    new_directory = "/bsuhome/tnde/scratch/felix/modis/blue_sky_albedo_new/"
    
    # Create the directory if it doesn't already exist
    Path(new_directory).mkdir(parents=True, exist_ok=True)
    
    # Construct the full output file path
    output_path = Path(new_directory + new_file_name)
    
    # # Only write the file if it doesn't already exist
    # if not output_path.is_file():
    #     # Write the blue sky albedo data to GeoTIFF using a reference format file
    #     write_to_raster(value, ex_format_tif, output_path)
    #     # print status
    #     print(f"Wrote to {output_path}")
    
    # Write the blue sky albedo data to GeoTIFF using a reference format file
    write_to_raster(value, ex_format_tif, output_path)
    # print status
    print(f"Wrote to {output_path}")

print("Done!!!")

## Visualize Black Sky and White Sky Albedo

In [ ]:
modis_albedo_data_dir_path = Path(modis_albedo_data_dir)

all_files = list(modis_albedo_data_dir_path.iterdir())
sorted_files = sorted(all_files, key=lambda x: x.name)

for file in sorted_files:
    if file.is_file():
        date = str(file).split(".")[1][1:]
        
        # Row, column, width, height
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7,2))
        with rio.open(file) as src:
            bsa_img = ax1.imshow(src.read(1))
            ax1.set_title(f"Black Sky Albedo-{date}")
            fig.colorbar(bsa_img, ax=ax1)
            
            wsa_img = ax2.imshow(src.read(2))
            ax2.set_title(f"White Sky Albedo-{date}")
            fig.colorbar(wsa_img, ax=ax2)
            # print(src.meta)
            
            plt.show()
            
            # Delay 1 second to allow for visual check

## Visualize AOD Data

In [ ]:
# # Define the path to the directory containing AOD raster files
# aod_data_dir_path = Path(aod_data_dir)

# # List and sort all files in the directory by filename
# all_files = list(aod_data_dir_path.iterdir())
# sorted_files = sorted(all_files, key=lambda x: x.name)

# # Loop through each sorted raster file
# for file in sorted_files:
#     if file.is_file():
#         # Extract the date string from the filename
#         # Assumes filename format like: something.D2022180.suffix.tif
#         date = str(file).split(".")[1][1:]

#         # Create a figure and axis for plotting (single subplot)
#         fig, ax1 = plt.subplots(1, 1, figsize=(7, 2))

#         # Open the AOD GeoTIFF file and read the first band
#         with rio.open(file) as src:
#             aod_img_data = src.read(1)

#             # Plot the AOD data as an image
#             aod_img = ax1.imshow(aod_img_data)
#             ax1.set_title(f"AOD Data - {date}")
#             fig.colorbar(aod_img, ax=ax1)

#             # Display the plot
#             plt.show()

#             # Delay 1 second to allow for visual check
#             # time.sleep(1)


## Visualize Blue Sky Albedo Data

In [ ]:
# Define the directory containing Blue Sky Albedo GeoTIFF files
blue_sky_albedo_path = Path("/bsuhome/tnde/scratch/felix/modis/blue_sky_albedo_new/")##_colorado/")  # Adjust path if needed

# List and sort all files in the directory by filename
all_files = list(blue_sky_albedo_path.iterdir())
sorted_files = sorted(all_files, key=lambda x: x.name)

# Loop through each sorted .tif file and visualize it
for file in sorted_files:
    if file.is_file():
        # Extract date string from filename
        # Assumes filename format like: "2022180_modis_blue_sky_albedo_new.tif"
        date = str(file.stem).split("_")[0]

        # Create a figure with one subplot (1 row, 1 column)
        fig, ax1 = plt.subplots(1, figsize=(7, 2))

        # Open the raster file and read the first band
        with rio.open(file) as src:
            blue_sky_albedo_data = src.read(1)

            # Plot the raster data
            img = ax1.imshow(blue_sky_albedo_data, cmap='viridis')
            ax1.set_title(f"Blue Sky Albedo Data - {date}")
            fig.colorbar(img, ax=ax1)

            # Display the plot
            plt.show()

            # Delay 1 second to allow for visual check
            # time.sleep(1)